<a href="https://colab.research.google.com/github/ymuto0302/base1_2021/blob/main/economicdata3_japan_stock_return.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 経済データの分析(3)：日経平均銘柄の騰落率の評価

## 日経平均銘柄とは？
東証第一部上場銘柄のうち，日本経済新聞社が選定した代表的な銘柄であり，225銘柄が選定されている。

それらの銘柄は，業種のバランスを考慮しつつ定期的に見直されており，これから得られる日経平均株価は日本の経済状況を知る上で重要な指標となっている。

詳細は下記の Wikipedia を参照のこと：

https://ja.wikipedia.org/wiki/%E6%97%A5%E7%B5%8C%E5%B9%B3%E5%9D%87%E6%A0%AA%E4%BE%A1#:~:text=%E6%97%A5%E7%B5%8C%E5%B9%B3%E5%9D%87%E6%A0%AA%E4%BE%A1%EF%BC%88%E3%81%AB%E3%81%A3%E3%81%91%E3%81%84%E3%81%B8,%E3%81%AB%E3%83%BC%E3%81%94%EF%BC%89%E3%81%A8%E3%82%82%E5%91%BC%E3%81%B0%E3%82%8C%E3%82%8B%E3%80%82&text=%E7%8F%BE%E5%9C%A8%E3%80%81%E6%9D%B1%E4%BA%AC%E8%A8%BC%E5%88%B8%E5%8F%96%E5%BC%95%E6%89%80,%E3%82%92%E5%AF%BE%E8%B1%A1%E3%81%AB%E3%81%97%E3%81%A6%E3%81%84%E3%82%8B%E3%80%82

## 騰落率とは？
騰落率とは，株価等の期間によって変動する値について，「比較したい一定の期間において，どのくらい上がったり下がったりしたか」を比率で表したものである。その定義は percent change と同一である。

**(注) 前回の授業では，変動パーセント (percentage change) として「前日からの変動率」を用いた。以下では，四半期単位での騰落率を調べる。**

(参考) 騰落率:指数用語解説:日経平均 読む・知る・学ぶ

https://indexes.nikkei.co.jp/atoz/2016/06/changeratio.html#:~:text=%E6%A0%AA%E4%BE%A1%E6%8C%87%E6%95%B0%E3%81%AE%E5%A0%B4%E5%90%88%E3%81%AF,%E3%81%AB100%E3%82%92%E4%B9%97%E3%81%98%E3%81%BE%E3%81%99%E3%80%82&text=%E9%A8%B0%E8%90%BD%E7%8E%87%E3%81%AF%E3%80%81%E7%9B%AE%E7%9A%84%E3%81%AB,%E5%B0%91%E3%81%97%E6%B3%A8%E6%84%8F%E3%81%8C%E5%BF%85%E8%A6%81%E3%81%A7%E3%81%99%E3%80%82


---

## (準備) Google Drive のマウント

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 日経平均銘柄データの読み込み
別途に準備した NIKKEI225_20201202.csv から企業名および株価コードを読み込む。

In [4]:
import pandas as pd
nikkei = pd.read_csv("/content/drive/My Drive/BASE/EconomicData/NIKKEI225_20201202.csv")

In [5]:
nikkei.head()

,業種,コード,社名
0,医薬品,4151,協和キリン
1,医薬品,4502,武田薬品工業
2,医薬品,4503,アステラス製薬
3,医薬品,4506,大日本住友製薬
4,医薬品,4507,塩野義製薬


### ticker symbol の生成 → データフレーム "nikkei" の拡張
後々，PandasDatareader を用いて stooq から株価データを取得するため，ticker symbols が必要となる。
そこで，このタイミングで生成しておく。

In [6]:
codes = list(nikkei['コード'])
ticker_symbols = [str(c)+'.JP' for c in codes] # "1234" → "1234.JP"
ticker_symbols = pd.DataFrame({'TickerSymbol':ticker_symbols}) # データフレームへ変換

# 元データと ticker symbols 列を結合
nikkei = pd.concat([nikkei, ticker_symbols], axis=1)

In [7]:
nikkei.head()

,業種,コード,社名,TickerSymbol
0,医薬品,4151,協和キリン,4151.JP
1,医薬品,4502,武田薬品工業,4502.JP
2,医薬品,4503,アステラス製薬,4503.JP
3,医薬品,4506,大日本住友製薬,4506.JP
4,医薬品,4507,塩野義製薬,4507.JP


## 日経平均銘柄に含まれる，業種ごとの企業数をチェック

In [8]:
# 業種毎の企業数を可視化
result = nikkei.groupby('業種').count()

import plotly.express as px
fig = px.bar(y=result['コード'], x=result.index, text=result['コード'], width=900, orientation='v')
fig['layout']['xaxis']['autorange'] = 'reversed' # これを付けないと横軸が（思ったのと）逆向きになる
fig.show()

---

## 株価データの取得
stooq から日本企業の株価を取得する。ここでは 2018年1月1日〜2021年4月30日の株価データを取得した。

ただし，stooq には１日あたりのアクセス数制限があるため，取得したデータを pickle 形式で保存しておく。

**(注意) 以下の「データ取得のためのコード」を実行しないこと**
→ **後に述べる「株価データの読み込み」を用いること**

In [ ]:
# PandasDatareader を用いた株価データの取得

from pandas_datareader import data
from datetime import datetime
import sys

start=datetime(2018, 1, 1)
end=datetime(2021, 4, 30)

# 業種(名)の取り出し
industries = set(nikkei['業種'])

# 業種毎に株価データを取得
stock = dict()
for industry in industries:
    print("=={}==".format(industry), file=sys.stderr)
    symbols = list(nikkei[nikkei['業種'] == industry]['TickerSymbol'])
    
    df = data.DataReader(symbols, data_source='stooq', start=start, end=end)['Close']
    
    stock[industry] = df
    
# stooq は１日あたりアクセス制限 (daily rate limit) があるため，
# 株価データを pickle 形式で保存する
import pickle
with open("/content/drive/My Drive/BASE/EconomicData/nikkei225_stock.pkl", "wb") as f:
    pickle.dump(stock, f)

==化学==
==パルプ・紙==
==造船==
==鉱業==
==銀行==
==その他金融==
==空運==
==窯業==
==繊維==
==その他製造==
==ガス==
==精密機器==
==海運==
==保険==
==機械==
==自動車==
==建設==
==小売業==
==サービス==
==電力==
==石油==
==陸運==
==証券==
==不動産==
==電気機器==
==通信==
==倉庫==
==商社==
==鉄鋼==
==食品==
==鉄道・バス==
==水産==
==医薬品==
==非鉄・金属==
==ゴム==


---

## 取得済み株価データの読み込み
上記のとおり，株価データを動的に取得すると "stooq の daily limit" による制限が掛るため，予めダウンロードしたデータを取り込む形とする。

**(注)株価データを Google Drive 上にて共有する。各自で自分のフォルダへコピーすること！**

In [9]:
# 株価データの読み込み
stock = pd.read_pickle('/content/drive/My Drive/BASE/EconomicData/nikkei225_stock.pkl')

# 月日の昇順に並べ替え
for key in stock:
    stock[key] = stock[key].sort_index()

変数 stock は辞書(dict)型になっており，そのキーは「業種」である。

各業種の値として，その業種に含まれる企業の株価データが格納されている。

In [11]:
# (参考) 変数 stock に含まれるキーを確認
stock.keys()

dict_keys(['化学', 'パルプ・紙', '造船', '鉱業', '銀行', 'その他金融', '空運', '窯業', '繊維', 'その他製造', 'ガス', '精密機器', '海運', '保険', '機械', '自動車', '建設', '小売業', 'サービス', '電力', '石油', '陸運', '証券', '不動産', '電気機器', '通信', '倉庫', '商社', '鉄鋼', '食品', '鉄道・バス', '水産', '医薬品', '非鉄・金属', 'ゴム'])

In [12]:
# (参考) 業種「機械」の値を表示
stock['機械']

Symbols,5631.JP,6103.JP,6113.JP,6301.JP,6302.JP,6305.JP,6326.JP,6361.JP,6367.JP,6471.JP,6472.JP,6473.JP,7004.JP,7011.JP,7013.JP
Date,,,,,,,,,,,,,,,
2018-01-04,3529.69,7238.83,1399.78,3868.21,4521.91,4199.21,2186.60,4235.24,13573.30,1626.81,515.60,1822.59,540.13,3871.51,3619.79
2018-01-05,3562.41,7304.48,1412.08,3865.50,4618.91,4228.85,2189.50,4310.45,13660.59,1656.51,524.67,1853.75,550.95,3890.34,3615.13
2018-01-09,3553.06,7407.63,1434.03,3928.79,4711.30,4293.07,2212.27,4352.75,13650.89,1652.00,529.22,1867.48,555.46,3884.06,3680.52
2018-01-10,3553.06,7323.23,1419.98,3977.59,4655.87,4288.13,2220.51,4296.35,13529.65,1666.40,532.84,1877.55,557.27,3886.76,3624.47
2018-01-11,3520.32,7435.76,1448.07,4018.28,4692.82,4386.94,2222.44,4277.54,13398.72,1676.30,533.76,1884.88,559.97,3873.30,3652.49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-22,2640.00,6010.00,1204.00,3251.00,3155.00,3275.00,2541.50,4775.00,22150.00,1051.00,327.00,1051.00,819.00,3215.00,2049.00
2021-04-23,2640.00,5950.00,1200.00,3148.00,3135.00,3265.00,2541.00,4720.00,21945.00,1033.00,319.00,1040.00,824.00,3198.00,2059.00
2021-04-26,2630.00,6020.00,1200.00,3191.00,3180.00,3285.00,2544.50,4755.00,22110.00,1017.00,317.00,1028.00,813.00,3225.00,2122.00


## 業種「機械」を例としてデータを眺めてみる

In [13]:
# ticker symbol では分かりにくいから，列名を社名へ変更
stock['機械'].columns = nikkei[nikkei['業種']=='機械']['社名']

In [15]:
stock['機械'].head()

社名,日本製鋼所,オークマ,アマダ,小松製作所,住友重機械工業,日立建機,クボタ,荏原製作所,ダイキン工業,日本精工,ＮＴＮ,ジェイテクト,日立造船,三菱重工業,ＩＨＩ
Date,,,,,,,,,,,,,,,
2018-01-04,3529.69,7238.83,1399.78,3868.21,4521.91,4199.21,2186.60,4235.24,13573.30,1626.81,515.60,1822.59,540.13,3871.51,3619.79
2018-01-05,3562.41,7304.48,1412.08,3865.50,4618.91,4228.85,2189.50,4310.45,13660.59,1656.51,524.67,1853.75,550.95,3890.34,3615.13
2018-01-09,3553.06,7407.63,1434.03,3928.79,4711.30,4293.07,2212.27,4352.75,13650.89,1652.00,529.22,1867.48,555.46,3884.06,3680.52
2018-01-10,3553.06,7323.23,1419.98,3977.59,4655.87,4288.13,2220.51,4296.35,13529.65,1666.40,532.84,1877.55,557.27,3886.76,3624.47
2018-01-11,3520.32,7435.76,1448.07,4018.28,4692.82,4386.94,2222.44,4277.54,13398.72,1676.30,533.76,1884.88,559.97,3873.30,3652.49


### 株価の移動平均
2020年3月中旬〜4月中旬にかけて，いずれの企業も一様に株価を下げている。
これは丁度，新型コロナウイルスが（日本国への）広がりを見せ，著名な芸能人の死去，および最初の緊急事態宣言が発出された時期である。

しかし，その後の「ダイキン工業」の株価の上がり方は半端ない。（何故だろうか？）

In [27]:
list(nikkei[nikkei['業種']=='機械']['社名'])

['日本製鋼所',
 'オークマ',
 'アマダ',
 '小松製作所',
 '住友重機械工業',
 '日立建機',
 'クボタ',
 '荏原製作所',
 'ダイキン工業',
 '日本精工',
 'ＮＴＮ',
 'ジェイテクト',
 '日立造船',
 '三菱重工業',
 'ＩＨＩ']

In [29]:
# 株価の移動平均
MA = stock['機械'].rolling(window=15).mean()

import plotly.express as px
# fig = px.line(MA) # Anaconda 用のコード
MA = MA.reset_index()
df_melt = MA.melt(id_vars='Date', value_vars=list(nikkei[nikkei['業種']=='機械']['社名']))
fig = px.line(df_melt, x="Date", y="value", color="社名", title="Movig Average of stock")
fig.show()


### 株価の変動パーセンテージの移動平均
15日間隔で percentage change をとり，さらに移動平均をとったが，2020年3月中旬〜4月中旬に全ての企業の株価が落ち込んだこと以外，特段の発見はなし。

In [32]:
# 株価の変動パーセンテージ(percentage change)の移動平均
change = stock['機械'].pct_change(15)
change = change.rolling(window=60).mean()

import plotly.express as px
#fig = px.line(change) # Anaconda 用のコード
change = change.reset_index()
df_melt = change.melt(id_vars='Date', value_vars=list(nikkei[nikkei['業種']=='機械']['社名']))
fig = px.line(df_melt, x="Date", y="value", color="社名", title="Movig Average of stock percentage change")
fig.show()

---

# ここから本番！

---

## 四半期ごとの騰落率からの分析
騰落率とは，株価等の期間によって変動する値について，「比較したい一定の期間において，どのくらい上がったり下がったりしたか」を比率で表したものである。その定義は percent change と同一である。

以下では，四半期として以下の期間を想定し，それぞれの期間における株価の騰落率を求めている。

- 2019年1〜3月
- 2019年4〜6月
- 2019年7〜9月
- 2019年10〜12月
- 2020年1〜3月
- 2020年4〜6月
- 2020年7〜9月
- 2020年10〜12月
- 2021年1〜3月

（例）「2019年1〜3月」の四半期の場合，1月1日と3月31日の株価（終値）を比較し，騰落率を求める。ただし，年始休業や土日曜日があるため，厳密には 1月4日と3月29日を比較することとなる。


In [44]:
from datetime import datetime

# 指定期間内での騰落率を求める
def pct_change(industry_stock, ticker_symbols, start, end):
    result = dict()
    
    for c in list(ticker_symbols):
        # 分析期間の限定
        condition = (start <= industry_stock.index) & (industry_stock.index <= end)
        
        # 該当する企業の株価データ
        d = industry_stock[condition][c]
        
        # 分析期間の初日と最終日で騰落率を求める
        # ここで曜日の影響を受けないよう，pandas の pct_change() を使わず，
        # 初日の株価 d[0] と最終日の株価 d[-1] から明示的に計算している
        result[nikkei[nikkei['TickerSymbol']==c]['社名'].values[0]] = (d[-1]-d[0]) / d[0]
    
    return result

def analysis(stock, industry=None):
    # 業種の絞り込み
    industry_stock = stock[industry] # 株価データ
    ticker_symbols = nikkei[nikkei['業種']==industry]['TickerSymbol'] # Ticker Symbpols

    result = dict()

    # 2019年1月〜2021年3月の各四半期の騰落率
    result['2019Q1'] = pct_change(industry_stock, ticker_symbols, datetime(2019,1,1), datetime(2019,3,31))
    result['2019Q2'] = pct_change(industry_stock, ticker_symbols, datetime(2019,4,1), datetime(2019,6,30))
    result['2019Q3'] = pct_change(industry_stock, ticker_symbols, datetime(2019,7,1), datetime(2019,9,30))
    result['2019Q4'] = pct_change(industry_stock, ticker_symbols, datetime(2019,10,1), datetime(2019,12,31))

    result['2020Q1'] = pct_change(industry_stock, ticker_symbols, datetime(2020,1,1), datetime(2020,3,31))
    result['2020Q2'] = pct_change(industry_stock, ticker_symbols, datetime(2020,4,1), datetime(2020,6,30))
    result['2020Q3'] = pct_change(industry_stock, ticker_symbols, datetime(2020,7,1), datetime(2020,9,30))
    result['2020Q4'] = pct_change(industry_stock, ticker_symbols, datetime(2020,10,1), datetime(2020,12,31))

    result['2021Q1'] = pct_change(industry_stock, ticker_symbols, datetime(2021,1,1), datetime(2021,3,31))

    # Pandas データフレームへの変換
    pcts = pd.DataFrame(result)
    
    # print(pcts)  # for DEBUG
    
    return pcts
    
# 株価データの読み込み
stock = pd.read_pickle('/content/drive/My Drive/BASE/EconomicData/nikkei225_stock.pkl')

# 2019年1月〜2021年3月の各四半期の騰落率を求める
pcts = analysis(stock, industry='機械')

# 可視化
# fig = px.line(pcts.T) # Anaconda 用のコード
pcts = pcts.T.reset_index()
df_melt = pcts.melt(id_vars='index', value_vars=list(nikkei[nikkei['業種']=='機械']['社名']))
fig = px.line(df_melt, x="index", y="value", color="variable", title="Movig Average of stock percentage change")
fig.show()

In [38]:
pcts

,index,日本製鋼所,オークマ,アマダ,小松製作所,住友重機械工業,日立建機,クボタ,荏原製作所,ダイキン工業,日本精工,ＮＴＮ,ジェイテクト,日立造船,三菱重工業,ＩＨＩ
0,2019Q1,-0.159292,-0.180968,-0.146844,-0.127342,-0.141742,-0.176370,-0.043456,-0.202887,-0.137749,-0.125301,-0.066635,-0.133649,-0.046163,-0.166715,0.041986
1,2019Q2,0.173992,0.097650,-0.065124,0.019036,-0.009450,0.081370,-0.102621,0.084896,-0.081051,0.111474,0.065611,0.075794,-0.126577,-0.011937,0.043515
2,2019Q3,-0.095554,-0.002794,0.048356,0.050641,0.167131,0.112476,0.109204,0.047968,0.018215,0.071330,0.052376,0.074934,0.131044,0.106923,0.093477
3,2019Q4,-0.006025,0.053449,-0.070409,-0.059992,0.028758,-0.202740,-0.053620,-0.131664,-0.084142,-0.088382,-0.066688,-0.014608,-0.135915,0.011063,-0.094019
4,2020Q1,0.584137,0.602945,0.409442,0.426435,0.542184,0.498401,0.225845,0.604279,0.140273,0.423004,0.740635,0.693429,0.129643,0.517159,0.987147
5,2020Q2,-0.190727,-0.310195,-0.080591,-0.238259,-0.179104,-0.301007,-0.196829,-0.217971,-0.263340,-0.197750,-0.162791,-0.168459,-0.142503,0.023398,-0.233633
6,2020Q3,-0.180977,-0.088158,-0.131065,-0.061690,-0.055694,-0.222105,-0.155792,-0.120295,-0.091969,-0.022470,0.071066,0.002439,-0.137390,0.070153,0.106810
7,2020Q4,-0.395121,-0.124783,-0.135927,-0.169656,-0.046347,0.273038,-0.179987,-0.163653,-0.179538,-0.104919,-0.252830,0.028714,-0.240346,-0.288022,-0.332682
8,2021Q1,0.155143,-0.105154,-0.103144,-0.180623,-0.180033,-0.165585,-0.112324,-0.260244,0.027239,-0.239357,-0.240469,-0.299115,-0.361899,-0.098542,-0.108686


上記のグラフから読み取れること：

- 2019年の間，いずれの企業も株価が変動は小さかった。
- 2020年Q1 (2020年1〜3月)は，いずれの企業の株価も上昇した。特に IHI の伸びが大きい。
- 2020年Q2 (2020年4〜6月)は，殆んどの企業の株価が落ち込んだ。
- パンデミックに伴う各国政府の金融緩和（＝大量の資金を株式市場へ流入させた）にも関わらず，2020年Q3以降は株価が上昇しない。金融緩和バブルは機械業界には効いていないのか？

## 課題
上記では業種を「機械」に絞って分析した。

皆さんは，日経平均銘柄のうち，医薬品(8銘柄)，鉄道・バス(8銘柄) および 化学工業(17銘柄) を対象に分析して欲しい。

社会情勢をふまえた上で仮説を立て，それが株価の変動，特に騰落率に反映されるか，を検討してください。

例えば，以下のような結論は得られるだろうか？

- 新型コロナウイルス・ワクチン開発を期待して医薬品業界の株価が上がる
- 巣ごもり消費の結果，運送業界（陸運・海運・空運）の株価が上がる
- 旅行や出張が手控えされた結果，鉄道・バス業界の株価が下がる
- 2020年は GOTOトラベルにより旅行者が増え，鉄道・バス業界の株価が上がる